In [6]:
import google.generativeai as genai
from langchain.embeddings.base import Embeddings
import numpy as np
import faiss
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
#from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS
#from langchain_classic.chains.combine_documents import create_stuff_documents_chain
#from langchain_classic.chains import create_retrieval_chain
import os
from dotenv import load_dotenv
from langchain_classic.memory import ConversationBufferMemory


load_dotenv()

True

In [7]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-2.5-flash')

In [9]:
import os
from langchain_community.document_loaders import Docx2txtLoader

folder_path = "C:\python\langchain\RAG\docs"
documents = []

for file in os.listdir(folder_path):
    if file.endswith(".docx"):
        file_path = os.path.join(folder_path, file)
        loader = Docx2txtLoader(file_path)
        docs = loader.load()
        documents.extend(docs)

<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
C:\Users\2425586\AppData\Local\Temp\ipykernel_26640\3468970197.py:4: SyntaxWarning: invalid escape sequence '\p'
  folder_path = "C:\python\langchain\RAG\docs"


In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
documents=text_splitter.split_documents(documents)
documents

In [ ]:
texts = [doc.page_content for doc in documents]
texts

In [19]:
#embeddings=configure.models.embed_content(model="gemini-embedding-001")

embeddings = genai.embed_content(
    model="models/text-embedding-004",
    content=texts)
embeddings["embedding"]

[[0.05980808,
  0.0075627165,
  -0.03626664,
  -0.034567136,
  -0.02936257,
  0.0507822,
  -0.013561508,
  -0.0034369277,
  -0.04148656,
  0.004270456,
  0.018091813,
  0.0043736957,
  0.041742235,
  -0.0010555304,
  -0.028292917,
  -0.025484614,
  -0.017096324,
  3.2908483e-05,
  -0.060573123,
  -0.03554614,
  -0.030457424,
  -0.02929937,
  -0.032679986,
  -0.0006054298,
  0.0047536884,
  -0.056404304,
  0.018899871,
  -0.018542964,
  0.0046910276,
  -0.070177905,
  -0.017169217,
  -0.020745581,
  0.0020313752,
  -0.05836783,
  -0.052506354,
  -0.05684565,
  -0.0017072882,
  -0.007175488,
  0.0123715,
  -0.06574144,
  -0.06154811,
  0.0034812568,
  -0.04673792,
  0.010409657,
  -0.043819536,
  0.06514547,
  -0.039192025,
  0.04928371,
  -0.04986341,
  0.0035459388,
  0.04003722,
  0.026789399,
  -0.047375467,
  -0.026223423,
  0.013573424,
  -0.039988723,
  -0.0066910745,
  -0.05137746,
  0.02357763,
  -0.021768093,
  0.0040289587,
  0.012405535,
  -0.022475986,
  -0.01682731,
  0.003

In [20]:
vec_np = np.array(embeddings["embedding"]).astype('float32')
vec_np.shape

(51, 768)

In [21]:
dimension = len(vec_np[0])
index = faiss.IndexFlatL2(dimension)
index.add(vec_np)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001B5FDACB690> >

Conversation memory

In [25]:
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history"
)

C:\Users\2425586\AppData\Local\Temp\ipykernel_26640\3749131566.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [26]:
# Load variables for use in prompts
variables = memory.load_memory_variables({})
print(variables["chat_history"])

[]


In [27]:
def embed(input_text):
    #embedding the input_text
    query_embedding = genai.embed_content(
    model="models/text-embedding-004",
    content=input_text)
    query_vector = np.array(query_embedding["embedding"]).astype('float32')
    D, I = index.search(np.array([query_vector]), 3)
    retireved_docs =[documents[i] for i in I[0]]
    final_docs=[]
    for doc in retireved_docs:
        final_docs.append(doc.page_content)
    context = "\n".join(final_docs)
    return context


In [28]:
while True:
    input_text = input("Enter your text here:")
    context = embed(input_text)

    if input_text.lower() == "exit" or input_text.lower() == "quit":
        break
    else:
        #prompt = f"you are an helpful AI assisstant, so respond accordingly {input_text}"
        prompt = f"""
        you are a helpful AI assisstant,so speak accordingly. 
        use the below context to answer the question.
        PREVIOUS MEMORY:{memory}
        CONTEXT: {context}
        QUESTION: {input_text}
        ANSWER:"""
        response = model.generate_content(prompt)
        print("User🙍🏻: ",input_text)
        #print("Context📚",context)
        print("AI🤖: ",response.text)
        memory.chat_memory.add_user_message(input_text)
        memory.chat_memory.add_ai_message(response.text)


User🙍🏻:  hi
AI🤖:  Hello! How can I assist you today?
User🙍🏻:  what is the purpose of Authentication & Security Issues 
AI🤖:  The purpose of the Authentication & Security Issues Policy is to outline procedures for managing authentication and security issues to protect customer accounts, ensure secure transactions, and comply with regulatory standards.
User🙍🏻:  tell me about rbi complaints
AI🤖:  RBI Complaints are defined as delays in failed transaction resolution or non-compliance that lead to customer complaints submitted to the RBI/BO Scheme.

The resolution protocol for RBI Complaints involves:
*   Registering the complaint, assigning it a priority, and acknowledging it within the defined Service Level Agreement (SLA).
*   Completing a root-cause analysis and developing a corrective action plan, which includes processing refunds or reversals according to guidelines.
User🙍🏻:  can you remember what we spoke
AI🤖:  Yes, I can! So far, we've discussed:

*   The purpose of the **Authentica